In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
#---Open the look-up-table made in water_vapor_optical_thickness
optical_thickness_df_07 = pd.read_pickle('tables/optical_thickness_band07')
optical_thickness_df_13 = pd.read_pickle('tables/optical_thickness_band13')
optical_thickness_df_14 = pd.read_pickle('tables/optical_thickness_band14')

#---Open the optical mass made in water_vapor_profile
optical_mass_df = pd.read_pickle('tables/optical_mass_example')

In [3]:
optical_thickness_df_07

,Optical Thickness
1000.0,5.284162e-05
975.0,4.399260e-05
950.0,4.131971e-05
925.0,3.872974e-05
900.0,7.043308e-05
850.0,5.670890e-05
800.0,4.809045e-05
750.0,4.099548e-05
700.0,2.732179e-05
650.0,1.050884e-05


Calculating expected radiance ($I_\lambda$):

$$
I_\lambda^\uparrow (\infty, \mu)= B_\lambda^\uparrow (T_\text{sfc}) e^{-\tau^*_\lambda / \mu} + \sum_{i=0}^N B_\lambda^\uparrow (z_i) \frac{d\mathfrak{T}_\lambda^\uparrow (z_i, \infty, \mu)}{dz} \Delta z_i
$$

* $B_\lambda^\uparrow (T_\text{sfc})$ is the blackbody radiance from the surface temperature
* $\tau^*$ is optical thickness of atmosphere
* $\mathfrak{T}_\lambda^\uparrow (z_i, \infty, \mu)$ is transmittance upward from height level ($z_i$) to TOA ($\infty$) with viewing angle ($\mu$)

Calculating the blackbody radiance ($B_\lambda^\uparrow$):

$$
B_\lambda (T) = \frac{2hc^2}{\lambda^5 [e^{hc/k\lambda T} -1]}
$$

* $h$ is the Planck constant, $6.626 \times 10^{-34} \text{ J Hz}^{-1}$
* $c$ is the speed of light, $3 \times 10^{8} \text{ m s}^{-1}$
* $k$ is the Boltzmann constant, $1.380 \times 10^{-23} \text{ J K}^{-1}$

In [4]:
def blackbody_radiance(T, wl):
    h = 6.626e-34
    c = 3e8
    k = 1.380e-23
    B = (2*h*c**2)/(wl**5 * np.exp((h*c)/(k*wl*T)-1))
    return B


In [5]:
T_sfc = optical_mass_df['Temperature'].loc[1000.0]
wl = 3.9e-6 #m
print("Wavelength:", wl*1e6, " um")

mu = 1
tau_star = optical_thickness_df_07['Optical Thickness'].sum()

sfc_component = blackbody_radiance(T_sfc, wl)*np.exp(-tau_star/mu)

print("Radiance from surface:", sfc_component, " W m-2 sr-1")

Wavelength: 3.9  um
Radiance from surface: 1510607.4786554466  W m-2 sr-1


In [6]:
I_levels = []
mu = 1

for i in range(len(optical_thickness_df_07)):
    T = optical_mass_df['Temperature'].iloc[i]
    B = blackbody_radiance(T, wl)
    tau_star = optical_thickness_df_07['Optical Thickness'].iloc[i:].sum()
    I_level = B*np.exp(-tau_star/mu)
    
    I_levels.append(I_level)

I = np.sum(I_levels)
print("Radiance from atmosphere: ", I, " W m-2 sr-1")

Radiance from atmosphere:  11421413.172018493  W m-2 sr-1
